In [1]:
import pandas as pd

# Scrape and save the table from wikipedia

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of the Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_stadiums_in_the_United_Kingdom_by_capacity'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

table = soup.find('table', {'class': 'wikitable'})

df = pd.read_html(str(table))[0]

# Get the Geo-codes for each stadium
- I used a google sheets tool and validated it

In [9]:
geo_df = pd.read_csv('/Users/kyle/syp-crime-py/scraping/wiki-stadiums-geocode.csv')

## clean the data


In [11]:
# Drop the 'Unnamed: 0' and 'Image' columns
geo_df = geo_df.drop(columns=['Unnamed: 0', 'Image'])

# Upload the complete crime data

In [3]:
crime_data = pd.read_csv('/Users/kyle/syp-crime-py/uk_street_crimes_data_2020_2023.csv')

In [7]:
len(crime_data)

19269992

# Function I used to create a filter which matched closest stadium in the main dataset
- once I matched sports area crimes, to closest stadium, I created a dictionary to collect the stadium lsoa names

In [ ]:

def match_stadium_to_lsoa(data):
    # Create a dictionary to store the mapping
    stadium_lsoa_mapping = {}
    
    for index, row in data.iterrows():
        stadium = row['Closest Stadium']
        lsoa_name = row['LSOA name']
        
        # Update the mapping
        stadium_lsoa_mapping[stadium] = lsoa_name
    
    return stadium_lsoa_mapping

# Apply the function
# mapping = match_stadium_to_lsoa(merged_df)

# Convert the dictionary to a DataFrame
# df_mapping = pd.DataFrame(list(mapping.items()), columns=['Stadium', 'LSOA Name'])

# Save the DataFrame to a CSV file
# df_mapping.to_csv('stadium_lsoa_mapping.csv', index=False)

# Fitler the dataset by LSOA where stadiums are
## We will use the last method as a filter

In [4]:
# Load the stadium_lsoa_mapping.csv into a DataFrame
stadium_lsoa_df = pd.read_csv('/Users/kyle/syp-crime-py/stadium_lsoa_mapping.csv')

# Extract the LSOA names from the stadium_lsoa_df
lsoa_list = stadium_lsoa_df['LSOA Name'].tolist()

# Filter the crime dataset based on the months and LSOA names
filtered_df = crime_data[crime_data['LSOA name'].isin(lsoa_list)]

# Save the filtered dataset to a new CSV file
filtered_df.to_csv('all_crime_in_stadium_lsoa_data.csv', index=False)

In [15]:
len(filtered_df)

150947

# Find all crimes within a radius of the Stadiums

In [16]:
import pandas as pd
import numpy as np

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

def closest_stadium(row, stadiums_df, max_distance_km):
    distances = stadiums_df.apply(lambda stadium: haversine(row['Longitude'], row['Latitude'], stadium['Longitude'], stadium['Latitude']), axis=1)
    closest_stadium_index = distances.idxmin()
    if distances[closest_stadium_index] <= max_distance_km:
        return stadiums_df.iloc[closest_stadium_index]['Stadium']
    else:
        return np.nan

# For 1 kilometer
filtered_1km_df = filtered_df.copy()
filtered_1km_df['Closest Stadium'] = filtered_1km_df.apply(lambda row: closest_stadium(row, geo_df, 1), axis=1)
filtered_1km_df.to_csv('lsoa_stadiums_1km_filled_gaps.csv', index=False)

# For 1 mile (approximately 1.60934 kilometers)
filtered_1mile_df = filtered_df.copy()
filtered_1mile_df['Closest Stadium'] = filtered_1mile_df.apply(lambda row: closest_stadium(row, geo_df, 1.60934), axis=1)
filtered_1mile_df.to_csv('lsoa_stadiums_1mile_filled_gaps.csv', index=False)

# For 2 miles (approximately 3.21869 kilometers)
filtered_2miles_df = filtered_df.copy()
filtered_2miles_df['Closest Stadium'] = filtered_2miles_df.apply(lambda row: closest_stadium(row, geo_df, 3.21869), axis=1)
filtered_2miles_df.to_csv('lsoa_stadiums_2miles_filled_gaps.csv', index=False)
